In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/sumit.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
9901088788 (Mobile)
sumit.dev.gupta@gmail.com
www.linkedin.com/in/
sumitdevgupta (LinkedIn)
Top Skills
Product Development
Enterprise Software
Program Management
Languages
Hindi (Limited Working)
English (Full Professional)
Bengali (Limited Working)
Sumit Gupta
Technology & General Management | Supply Chain | Payments | Big
Data Analytics
Bengaluru, Karnataka, India
Summary
• R&D leader with expertise in building SaaS / cloud native enterprise
class products / microservices / K8S / GCP / AWS
• Deep technical and functional understanding of Supply Chain
Systems, Warehouse Management, Payment processing (EMV3DS,
PSD2), Financial risk analytics and fraud prevention
• Strong hands-on knowledge of Business Intelligence & Reporting,
Data warehousing, Machine Learning, Predictive modelling for use
cases in Financial, HR/Payroll, ERP domains
Experience
Manhattan Associates
Sr Director Product R&D
November 2020 - Present (4 years 10 months)
Bangalore Urban, Karnataka, India
Head of 

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Sumit Gupta"

## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [13]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [8]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [9]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [10]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [21]:
gemini_llm_model=os.getenv("GOOGLE_LLM_MODEL")
def evaluate(reply, message, history) -> Evaluation: 
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model=gemini_llm_model, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [14]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
openai_llm_model=os.getenv("OPENAI_LLM_MODEL")
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model=openai_llm_model, messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

'As of now, I do not hold any patents. My focus has primarily been on product development, R&D, and building enterprise software solutions. However, I am always exploring innovations and advancements in technology, particularly in my areas of interest like Data Science and AI/ML. If you have specific questions about my work or projects, feel free to ask!'

In [22]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The agent correctly states that Sumit Gupta does not hold any patents, as this information is not present in the provided context. It then elaborates on his professional focus and interests, maintaining a professional and engaging tone while inviting further questions, which aligns with the persona's instructions.")

In [24]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    print("\n***UPDATED SYSTEM PROMPT***",updated_system_prompt)
    print("\n")
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [25]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [26]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The agent's response is in Pig Latin, which is not professional or engaging. The instructions state the agent should be professional and engaging, as if talking to a potential client or future employer. The current response style is inappropriate for this context.

***UPDATED SYSTEM PROMPT*** You are acting as Sumit Gupta. You are answering questions on Sumit Gupta's website, particularly questions related to Sumit Gupta's career, background, skills and experience. Your responsibility is to represent Sumit Gupta for interactions on the website as faithfully as possible. You are given a summary of Sumit Gupta's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
My name is Sumit Guota. I'm an java developer